In [1]:
import pandas as pd
import numpy as np
import json
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import precision_score, recall_score, accuracy_score, mean_squared_error

In [2]:
# Return one season's financial news dataframe
def season_dataframe(ticker, season):
    with open('./financial_news/'+ticker+str(season), 'r') as ticker_news:
        content  = ticker_news.read()
    records = json.loads(content)
    df = pd.DataFrame(records)
    return df

In [3]:
# Create dataframe for each ticker, storing its news
#ALL_TICKERS = ["$TSLA", "$NVDA", "$INTC", "$PFE", "$SPGI", "$LRCX", "$TMUS", "$ADSK", "$VRTX", "$TWTR", "$EBAY", "$CARR", "$VRSN", "$GRMN", "$ANET", "$AAL"]
ALL_TICKERS = ["$TSLA", "$NVDA", "$INTC", "$PFE", "$SPGI", "$LRCX", "$TMUS", "$ADSK", "$VRTX", "$TWTR", "$EBAY", "$CARR", "$GRMN", "$ANET", "$AAL"]
ALL_TICKERS = ["$TSLA"]

df = []
for ticker in ALL_TICKERS:
  # Concatenate 8 seasons into one dataframe
  df_temp = pd.DataFrame()
  for i in range(1, 2):
      new_df = season_dataframe(ticker, i)
      df_temp = pd.concat([df_temp, new_df], ignore_index=True)
  df.append(df_temp)

In [6]:
df[0]

,symbol,publishedDate,title,image,site,text,url
0,TSLA,2020-12-31 23:04:00,Tesla to deliver China-made Model Y SUVs this ...,https://cdn.snapi.dev/images/v1/5/m/m02d202101...,Reuters,Tesla Inc said on Friday it has started sellin...,https://www.reuters.com/article/us-tesla-china...
1,TSLA,2020-12-31 21:44:01,2020: Several Chinese Stocks Outperformed Thei...,https://cdn.snapi.dev/images/v1/f/j/catalog-ma...,Seeking Alpha,2020: Several Chinese Stocks Outperformed Thei...,https://seekingalpha.com/article/4396892-2020-...
2,TSLA,2020-12-31 16:39:55,EV Company News For The Month Of December 2020,https://cdn.snapi.dev/images/v1/l/r/sssik22-c5...,Seeking Alpha,Global electric car sales records for November...,https://seekingalpha.com/article/4396884-ev-co...
3,TSLA,2020-12-31 16:34:55,"Tesla, Volkswagen, Renault See Strong Share In...",https://cdn.snapi.dev/images/v1/v/x/s3xy-14.jpg,Benzinga,The European market continues to see strong ad...,https://www.benzinga.com/news/20/12/18973120/t...
4,TSLA,2020-12-31 16:29:13,Tech's top seven companies added $3.4 trillion...,https://cdn.snapi.dev/images/v1/s/t/stocks23-1...,CNBC,"Big Tech got much bigger in 2020, and Tesla jo...",https://www.cnbc.com/2020/12/31/techs-top-seve...
...,...,...,...,...,...,...,...
973,TSLA,2020-10-26 23:55:08,Tesla Ends Controversial Indemnification Agree...,https://cdn.snapi.dev/images/v1/x/1/105843585-...,Benzinga,Tesla Inc (NASDAQ: TSLA) has ended a controver...,https://www.benzinga.com/news/20/10/18066423/t...
974,TSLA,2020-10-26 09:20:00,"Tesla Will Export 7,000 Cars From China to Eur...",https://cdn.snapi.dev/images/v1/o/v/te7.jpg,The Motley Fool,The Shanghai-made Model 3s will be delivered t...,https://www.fool.com/investing/2020/10/26/tesl...
975,TSLA,2020-10-26 08:04:28,"For Now, Tesla Stock Is Too Wild a Ride",https://cdn.snapi.dev/images/v1/a/c/m02d202006...,InvestorPlace,Despite meteoric gains this year and five stra...,https://investorplace.com/2020/10/for-now-tesl...
976,TSLA,2020-10-26 00:08:52,7 Strong Nasdaq Stocks to Buy As They Keep Mar...,https://cdn.snapi.dev/images/v1/r/8/vitaldata1...,InvestorPlace,Most Nasdaq stocks have seen ups and downs thi...,https://investorplace.com/2020/10/7-strong-nas...


In [8]:
# test
df[0] = df[0].head(10)

In [9]:
df[0]

,symbol,publishedDate,title,image,site,text,url
0,TSLA,2020-12-31 23:04:00,Tesla to deliver China-made Model Y SUVs this ...,https://cdn.snapi.dev/images/v1/5/m/m02d202101...,Reuters,Tesla Inc said on Friday it has started sellin...,https://www.reuters.com/article/us-tesla-china...
1,TSLA,2020-12-31 21:44:01,2020: Several Chinese Stocks Outperformed Thei...,https://cdn.snapi.dev/images/v1/f/j/catalog-ma...,Seeking Alpha,2020: Several Chinese Stocks Outperformed Thei...,https://seekingalpha.com/article/4396892-2020-...
2,TSLA,2020-12-31 16:39:55,EV Company News For The Month Of December 2020,https://cdn.snapi.dev/images/v1/l/r/sssik22-c5...,Seeking Alpha,Global electric car sales records for November...,https://seekingalpha.com/article/4396884-ev-co...
3,TSLA,2020-12-31 16:34:55,"Tesla, Volkswagen, Renault See Strong Share In...",https://cdn.snapi.dev/images/v1/v/x/s3xy-14.jpg,Benzinga,The European market continues to see strong ad...,https://www.benzinga.com/news/20/12/18973120/t...
4,TSLA,2020-12-31 16:29:13,Tech's top seven companies added $3.4 trillion...,https://cdn.snapi.dev/images/v1/s/t/stocks23-1...,CNBC,"Big Tech got much bigger in 2020, and Tesla jo...",https://www.cnbc.com/2020/12/31/techs-top-seve...
5,TSLA,2020-12-31 15:35:15,Tesla Hits Records as Nasdaq's Big 2020 Winner...,https://cdn.snapi.dev/images/v1/n/k/urlhttps3a...,The Motley Fool,High-flying stocks continued to hang onto thei...,https://www.fool.com/investing/2020/12/31/tesl...
6,TSLA,2020-12-31 13:52:22,XPEV Is Worth Far More Than Its Present Price,https://cdn.snapi.dev/images/v1/t/w/auto20.jpg,InvestorPlace,XPeng is worth far more than its present price...,https://investorplace.com/2020/12/xpev-stock-x...
7,TSLA,2020-12-31 13:33:25,Tesla: Not an Automatic Addition to the S&P 50...,https://cdn.snapi.dev/images/v1/4/a/tesla-not-...,ETF Trends,"By Daniel Perrone, Director and Head of Operat...",https://www.etftrends.com/esg-channel/tesla-no...
8,TSLA,2020-12-31 12:22:38,"From Tesla to PayPal, Carnival to Occidental, ...",https://cdn.snapi.dev/images/v1/f/r/from-tesla...,Business Insider,US stocks posted strong returns in 2020 despit...,https://www.businessinsider.com/tesla-paypal-p...
9,TSLA,2020-12-31 11:59:11,Why this investor doesn't see the 'luster' com...,https://cdn.snapi.dev/images/v1/s/g/why-this-i...,CNBC Television,"CNBC's ""Squawk Alley"" discusses what may be ne...",https://www.youtube.com/watch?v=pW1rc3uepAo


# Fetch url column

In [10]:
url_df = []
for d in df:
  url_df.append(d[['symbol', 'url']])

In [11]:
from bs4 import BeautifulSoup
import urllib
import re

# Fetch body content

In [12]:
body_content = [[]]

In [13]:
# Fetch body content of news
def fetchContent(url):
  try:
    webUrl = urllib.request.urlopen(url)
  except:
    return ""
  if (webUrl.getcode() == 200):
    content = webUrl.read()
    soup = BeautifulSoup(content, 'lxml')
    table = soup.findAll('p')
    content = ""
    for index, text in enumerate(table):
      original = str(text)
      modified = re.sub("^<p[^>]*>", "", original)
      modified = re.sub("</p>$", "", modified)
      modified = re.sub("<.[^>]*>", "", modified)
      modified = re.sub("</.>", "", modified)
      content+=modified
    print(content)
    return content
  else:
    return ""

In [14]:
for ticker in url_df:
  ticker['body'] = ticker['url'].apply(fetchContent)

Discover Thomson ReutersBy Reuters Staff2 Min ReadBEIJING (Reuters) -Tesla Inc said on Friday it has started selling China-made Model Y sport-utility vehicles (SUV) and will deliver them to customers this month, as the U.S. electric vehicle maker expands sales in the world’s biggest car market.China, which offers hefty subsidies for electric vehicles as it seeks to cut down on pollution from petrol or diesel cars, is key to Tesla’s global strategy. It is expanding its Shanghai car factory, where it also builds its Model 3 sedans. In October, it started exporting Model 3 vehicles to Europe.It is also adding manufacturing capacity for EV chargers in Shanghai and expanding its sales and service network around the country. It sold over 20,000 vehicles in November.The starting price of a Model Y in China is now 339,900 yuan ($52,091.95), according to its China website.Tesla’s rivals in China include Volkswagen AG, BMW as well as local startups such as Nio Inc, Xpeng Inc, and Li Auto Inc, wh

/Users/vincent/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [15]:
url_df[0]

,symbol,url,body
0,TSLA,https://www.reuters.com/article/us-tesla-china...,Discover Thomson ReutersBy Reuters Staff2 Min ...
1,TSLA,https://seekingalpha.com/article/4396892-2020-...,
2,TSLA,https://seekingalpha.com/article/4396884-ev-co...,
3,TSLA,https://www.benzinga.com/news/20/12/18973120/t...,The European market continues to see strong ad...
4,TSLA,https://www.cnbc.com/2020/12/31/techs-top-seve...,In this articleTech's biggest companies just w...
5,TSLA,https://www.fool.com/investing/2020/12/31/tesl...,Returns as of 7/14/2021Returns as of 7/14/2021...
6,TSLA,https://investorplace.com/2020/12/xpev-stock-x...,
7,TSLA,https://www.etftrends.com/esg-channel/tesla-no...,"By Daniel Perrone, Director and Head of Operat..."
8,TSLA,https://www.businessinsider.com/tesla-paypal-p...,
9,TSLA,https://www.youtube.com/watch?v=pW1rc3uepAo,


In [ ]:
webUrl = urllib.request.urlopen(url_df.url.loc[0])

In [ ]:
# result code
webUrl.getcode()

In [ ]:
content = webUrl.read()

In [ ]:
soup = BeautifulSoup(content, 'lxml')

In [ ]:
import re

In [ ]:
table = soup.findAll('p')
for i in table:
  print(i)
  print("***********")

In [ ]:
for index, text in enumerate(table):
  original = str(text)
  modified = re.sub("^<p[^>]*>", "", original)
  modified = re.sub("</p>$", "", modified)
  table[index] = modified
  print(modified)

In [ ]:
for i in table:
  print(i)

In [ ]:
content = ""
for index, text in enumerate(table):
  original = str(text)
  modified = re.sub("<.[^>]*>", "", original)
  modified = re.sub("</.>", "", modified)
  content+=modified
  print(modified)

In [ ]:
content